# Projet TAL M1 S1 : POS Tagging
structure du projet expliqué ici : https://u-paris.zoom.us/rec/share/rFB-gp7vkgMK31kfPxEsyNF2k6q7mHcq3P9MQiovimkr9ebIewZpuD62RBk6SmM.31Mmzx1VyMqQXRtu?startTime=1607605609000

# Implémentation du classifieur

## Import du corpus
Cours Zoom du 17/12 - 35 min

On charge les trois corpus *in-domain* sous la forme de listes de dictionnaires : chaque phrase a une clé "mots" qui est associée à une liste des mots, et une clé "POS" associée à une liste de POS. 

On utilise les trois corpus distincts de French-GSD :
- Train : apprentissage.
- Dev : validation. Pour tester et améliorer le modèle. 
- Test : évaluation. On ne l'utilisera pas pendant l'apprentissage ou le test.

In [1]:
# clés des dictionnaires
mot = "mot"
gold = "gold_label"
mot_prec = "préc"
mot_suiv = "suiv"
maj = "maj"
num = "nb"
nonAlphanum = "non alphanum"
long = "longueur"

In [2]:
def load_corpus(file):
    with open(file, "r") as f:
        content = f.read() # chargement du corpus
    content = content.split("\n\n") # séparation en phrases
    corpus = []
    for phrase in content: # pour chaque phrase
        phrase_dico = {"mots" : [], "gold_labels" : []} # liste qui contiendra 1 dictionnaire par mot de la phrase
        for line in phrase.splitlines():
            if not line.startswith("#"): # on ignore les lignes qui commencent par #
                features = line.split("\t")
                phrase_dico["mots"].append(features[1])
                phrase_dico["gold_labels"].append(features[3])
        corpus.append(phrase_dico)
    return corpus

gsd_train = load_corpus("corpus-in-domain/fr_gsd-ud-train.conllu")
gsd_test = load_corpus("corpus-in-domain/fr_gsd-ud-test.conllu")
gsd_dev = load_corpus("corpus-in-domain/fr_gsd-ud-dev.conllu")

In [3]:
print("---- Aperçus d'une phrase de chaque corpus-----", end="\n\n")
print(gsd_train[1], end="\n\n")
print(gsd_test[102], end="\n\n")
print(gsd_dev[564])

---- Aperçus d'une phrase de chaque corpus-----

{'mots': ["L'", 'œuvre', 'est', 'située', 'dans', 'la', 'galerie', 'des', 'de', 'les', 'batailles', ',', 'dans', 'le', 'château', 'de', 'Versailles', '.'], 'gold_labels': ['DET', 'NOUN', 'AUX', 'VERB', 'ADP', 'DET', 'NOUN', '_', 'ADP', 'DET', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT']}

{'mots': ['La', 'gestion', 'et', "l'", 'exploitation', 'de', 'la', 'salle', 'de', 'concert', 'Wagram', ',', 'récemment', 'rénovée', ',', 'sera', 'assurée', 'par', 'Eurosites', ',', 'leader', 'en', 'France', 'de', 'la', 'location', 'de', 'salles', '.'], 'gold_labels': ['DET', 'NOUN', 'CCONJ', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'PROPN', 'PUNCT', 'ADV', 'VERB', 'PUNCT', 'AUX', 'VERB', 'ADP', 'PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'PUNCT']}

{'mots': ['Cette', 'espèce', 'est', 'endémique', 'du', 'de', 'le', 'département', 'de', 'Nariño', 'en', 'Colombie', '.'], 'gold_labels': ['DET'

## Extraction des caractéristiques

La fonction feature_extraction renvoie une liste de dictionnaires (un par mot) qui contiennent les caractéristiques suivantes pour chaque mot :
- mot
- mot précédent : pour le premier mot de la phrase son mot précédent sera "START", ce qui permettra de prendre en compte la caractéristique "être le premier mot".
- mot suivant : pour le dernier mot, ce sera "END".
- commence par une lettre majuscule
- est numérique
- contient des caractères non alphanumériques
- longueur du mot : redéfinir 3 caractéristiques booléens/binaires : a 1 seule caractère, a moins de 3 caractères, a plus de 3 caractères.
- a un suffixe nominal
- a un suffixe adjectival
- a un suffixe verbal (ça marche que sur les lemmes !)
- a un suffixe adverbial (là ça marche sur la forme fléchie)

Met-on dans les listes les suffixes qui appartiennent à 2 catégories ?
Peut-on utiliser les lemmes ?

On pourrait ajouter des suffixes aussi. Par exemple une feature pourrait être "finir en -ment", une autre suffi_nominaux (-al, -ion, etc.), etc.

On ne conserve plus la structure des phrases, qui n'est plus nécessaire une fois qu'on a extrait les informations comme mot précédent et mot suivant.

In [4]:
def feature_extraction(corpus):

    corpus_features = []

    for phrase in corpus: # ajout des features additionnelles
        for prev, word, suiv in zip(["START"] + phrase["mots"][:-1], phrase["mots"], phrase["mots"][1:] + ["END"]):
            # création de triplets (mot précédent, mot, mot suivant)
            # avec "START" en prev pour le 1er mot
            # et "END" en suiv pour le dernier

            # ajout d'un dictionnaire de features du mot au corpus
            corpus_features.append({mot : word, 
                gold : phrase["gold_labels"][phrase["mots"].index(word)], # on récupère le gold_label correspondant
                mot_prec : prev,
                mot_suiv : suiv,
                num : word.isnumeric(),
                nonAlphanum : not word.isalnum(),
                long : len(word)})

    return corpus_features # renvoie les features transformés en vecteurs one-hot


gsd_train_features = feature_extraction(gsd_train)
gsd_dev_features = feature_extraction(gsd_dev)
gsd_test_features = feature_extraction(gsd_test)

In [5]:
print(*gsd_train_features[3:6], sep="\n")
print(len(gsd_train_features))

{'mot': 'sont', 'gold_label': 'AUX', 'préc': 'cérébrales', 'suiv': 'devenu', 'nb': False, 'non alphanum': False, 'longueur': 4}
{'mot': 'devenu', 'gold_label': 'VERB', 'préc': 'sont', 'suiv': 'si', 'nb': False, 'non alphanum': False, 'longueur': 6}
{'mot': 'si', 'gold_label': 'ADV', 'préc': 'devenu', 'suiv': 'courantes', 'nb': False, 'non alphanum': False, 'longueur': 2}
364349


A SUPPRIMER PLUS TARD, ça ne sert à rien [ne surtout pas faire tourner]

In [6]:
def make_all_one_hots(corpus_features):
    '''Renvoie un dictionnaire qui associe à chaque mot du corpus un vecteur one hot'''
    
    # création liste de mots
    all_words = set()
    for item in corpus_features:
        all_words.add(item[mot].lower()) # on ne prend pas en compte les majuscules pour les vecteurs one hot

    all_words.update({"start", "end"}) # ajout de START et END, qui ne sont pas dans les mots

    all_words = list(all_words) # transformation du set en liste pour conserver l'ordre

    all_one_hots = {}
    i = 0
    for word in all_words:
        vec = [0]*len(all_words)
        vec[i] = 1
        i+= 1
        all_one_hots[word] = vec

    return all_one_hots


def make_one_hot_features(corpus_features, all_one_hots):
    '''Transforme dictionnaire de features en vecteur one hot'''

    vecteurs = []

    count = 0

    for token in corpus_features:
        count += 1
        print(count)

        vec_item = []

        for key, value in token.items(): # pour chaque feature de notre mot
            if isinstance(value, str) and not key == gold: # si la valeur est une chaîne de caractère > encodage one hot   
                # cette méthode est affreusement lente....

                #for word in all_words:
                #    if word == item[mot]:
                #        vec_item.append(1)
                #    else:
                #        vec_item.append(0)

                # deuxième méthode, peut-être plus rapide
                #one_hot = [0]*len(all_words) # on initialise une liste de zéros de longueur du vocabulaire
                
                #try :
                #    one_hot[all_words.index(value.lower())] = 1 # 1 à la place correspondant au mot
                #except:
                #    pass

                #vec_item += one_hot # on concatène la liste one hot


                # troisième méthode...
                try:
                    vec_item += all_one_hots.get(value)
                except:
                    pass

            elif isinstance(value, bool): # si c'est un booléen, append après l'avoir converti en entier
                vec_item.append(int(value))
            elif isinstance(value, int): # si c'est un entier, append
                vec_item.append(value)
        
        vecteurs.append(vec_item)
    
    return vecteurs
    

#gsd_train_vec = make_one_hot_features(gsd_train_features)
gsd_train_all_one_hots = make_all_one_hots(gsd_train_features)

In [15]:
# print(*zip(gsd_train_features,gsd_train_vec))
print(list(gsd_train_all_one_hots.items())[-1])

 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## Implémentation de l'algorithme de classification
On a choisi d'implémenter la classification avec un perceptron moyenné.

Etiquettes du corpus : ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X

Sources : cours 3/12 + perceptron.pdf + zoom du 17/12

Je comprends pas comment on doit gérer les mots : comment on peut les ajouter ou les enlever de nos vecteurs ?!

In [8]:
def perceptron_train(training_set, MAX_EPOCH=10):
    
    tags = ["ADJ", "ADP", "ADV", "AUX", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "PUNCT", "SCONJ", "SYM", "VERB", "X"]

    # TODO initialisation des poids : à zéros ou bien de manière aléatoire ??
    a = [[0]*len(training_set[0]) for tag in tags] # poids moyennés
    # un vecteur de poids par classe - plutôt un dictionnaire ? FIXME

    for i in range(0, MAX_EPOCH):
        w = [[0]*len(training_set[0]) for tag in tags] # initialisé du vecteur de poids
        shuffled_set = shuffle(training_set.copy()) # on mélange le training set

        for phrase in shuffled_set:
            for word in phrase:
                prediction = max()
                # on cherche le tag le plus probable... mais comment ?... 
                
                # on compare avec le gold_label
                # si mauvaise prédiction
                    #  on ajoute x(i) à chaque poids de l'étiquette correcte
                    #  on retire x(i) à chaque poids de l'étiquette mal prédite

        # on ajoute w à a
    return a

poids_gsd_train = perceptron_train(gsd_train_features)

NameError: name 'shuffle' is not defined

## Evaluation des performances sur le corpus de validation (Dev)
Il faudra bien tester les hyper paramètres (epoch...)

## Evaluation sur le corpus Test

# Evaluation hors-domaine

## Analyse de l'impact du changement de domaine
* identifier causes de la baisse de performance : analyse de sortie, matrice de confusion, erreurs les + fréquentes
* réfléchir à des caractéristiques plus adaptées
* sélection d'un nouvel ensemble d'apprentissage avec des exemples représentatif (généré par un modèle de langue type TP2)

### Corpus UGC (User-generated content)

### Corpus littéraire

## Développement de systèmes robutes au changement de domaine